In [47]:
import pandas as pd
import pandas as pd
import re
import string
import nltk
nltk.download('wordnet')

import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import operator
from nltk.corpus import stopwords
stop=[]
stop.extend(stopwords.words('english'))
from textblob import TextBlob

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()


[nltk_data] Downloading package wordnet to /home/prashant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/prashant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/prashant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [48]:
nltk.download('brown')

from nltk.corpus import brown
word_list = brown.words()
word_set = set(word_list)

def spelling_mistakes_check(sentences):
    l = []
    for i in sentences:
        
        cnt = 0
        words = i.split(' ')
        for w in words:
            if w not in word_set:
                cnt +=1
        l.append(cnt)
    return l

[nltk_data] Downloading package brown to /home/prashant/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [49]:
from string import punctuation

def get_count_of_puncs(essay_list):
    puncs = list(punctuation)
    punc_count_list = []
    for essay in essay_list:
        c = 0
        for punc in puncs:
            if punc in essay:
                c = c +1
        punc_count_list.append(c)
    return punc_count_list

def get_number_of_char(essay_list):
    num_char = []
    for essay in essay_list:
        num_char.append(len(essay))
    return num_char

def word_to_sent_ratio(essay_list):
    w_s = []
    for essay in essay_list:
        num_sent = len(essay.split('.'))
        num_word = len(essay.split(' '))
        if num_word == 0:
            w_s.append(0)
        else:
            w_s.append(num_sent/(num_word*1.0))
    return w_s

def get_avg_word_length_per_essay(essay_list):
    avg = []
    for essay in essay_list:
        c = []
        for w in essay.split():
            c.append(len(w))
        
        if len(c) == 0:
            avg.append(0)
        else:
            avg.append(sum(c)/len(c))
    return avg

In [57]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

def sentence_to_wordlist(raw_sentence):
    
    clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)
    
    return tokens

# tokenizing an essay into a list of word lists
def tokenize(essay):
    stripped_essay = essay.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokenized_sentences.append(sentence_to_wordlist(raw_sentence))
    
    return tokenized_sentences

def count_lemmas(essay):
    
    tokenized_sentences = tokenize(essay)      
    
    lemmas = []
    wordnet_lemmatizer = WordNetLemmatizer()
    
    for sentence in tokenized_sentences:
        tagged_tokens = nltk.pos_tag(sentence) 
        
        for token_tuple in tagged_tokens:
        
            pos_tag = token_tuple[1]
        
            if pos_tag.startswith('N'): 
                pos = wordnet.NOUN
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('J'):
                pos = wordnet.ADJ
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('V'):
                pos = wordnet.VERB
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('R'):
                pos = wordnet.ADV
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            else:
                pos = wordnet.NOUN
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
    
    lemma_count = len(set(lemmas))
    
    return lemma_count

def count_lemma_for_all(essay_list):
    lemma_count_list=[]
    for essay in essay_list:
        lemma_count_list.append(count_lemmas(essay))
    return lemma_count_list

In [50]:
def get_ner_tags(sentences):
    all_sample_tags = []
    for i in sentences:
        dict_count = {}
        doc = nlp(i.decode('utf8'))
        result = ([(X.label_) for X in doc.ents])

        for tag in result:
            if tag not in dict_count:
                dict_count[tag] = 1
            else:
                dict_count[tag]  += 1
        all_sample_tags.append(dict_count)
    
    cc = pd.DataFrame.from_dict(all_sample_tags)
    cc.fillna(0,inplace=True)
    return cc

In [51]:
def tfidf(text):
    from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
    vectorizer = CountVectorizer(min_df=20)
    vectorizer.fit(text)
    # encode document
    vector = vectorizer.transform(text)
    p = pd.DataFrame(vector.toarray())
    return p

In [37]:
#Function so that one session can be called multiple times. 
#Useful while multiple calls need to be done for embedding. 
import tensorflow as tf
import tensorflow_hub as hub
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

embed_fn = embed_useT('/home/prashant/google_sen_vec/')


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0427 02:09:14.617511 140315913066240 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


I0427 02:09:14.919055 140315913066240 monitored_session.py:222] Graph was finalized.


INFO:tensorflow:Running local_init_op.


I0427 02:09:15.743639 140315913066240 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0427 02:09:16.883847 140315913066240 session_manager.py:493] Done running local_init_op.


In [78]:
def get_part_of_speech_tags(sentences):
    all_sample_tags = []
    for i in sentences:
        dict_count = {}
        result = (TextBlob(i))
        for words, tag in result.tags:
            if tag not in dict_count:
                dict_count[tag] = 1
            else:
                dict_count[tag]  += 1
        all_sample_tags.append(dict_count)
    
    cc = pd.DataFrame.from_dict(all_sample_tags)
    cc.fillna(0,inplace=True)
    return cc




def get_sen_len(sentences):
    sen_len = []
    for i in sentences:
        i = len(i.split(' '))
        sen_len.append(i)
    return sen_len


def get_noun_phrase(sentences):
    l = []
    for i in sentences:
        doc = nlp(i)
        cnt = 0
        for chunk in doc.noun_chunks:
            cnt +=1
        l.append(cnt)
    return l 

def pca(X):
    import numpy as np
    from sklearn.decomposition import PCA
    pca = PCA(n_components=100)
    pc = pca.fit_transform(X)
    return pc

def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(str.maketrans('', '', string.punctuation))
    filter_str = final_str#.translate(str.maketrans('', '', digits))
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stop:
            str_ += ' '  + (w)
    
    return str_.strip()


def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:   
                get_word_vec = model[w]
            except:
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen


#vocab = model.keys()

df_test = pd.read_csv('test_dataset.csv')
df_test.head()
df_train = pd.read_csv('train_dataset.csv')

df_train['source_'] = 1
df_test['source_'] =0

df = pd.concat([df_train,df_test])
df['score_1'].fillna(1, inplace = True)
df['score_2'].fillna(1, inplace = True)
df['score_3'].fillna(1, inplace = True)
df['score_4'].fillna(1, inplace = True)
df['score_5'].fillna(1, inplace = True)

df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)

multi_datasets = {}
groups = df.groupby('Essayset')
for name, group in groups:
    multi_datasets[name] = group

pp = []
def different_models(model,multi_datasets):
    flag = 1
    for i in multi_datasets:
        print(i)
        
        single_df = multi_datasets[i]
        single_df.reset_index(inplace= True)
        single_df.dropna(axis=0,inplace=True)

        test_id = list(single_df.loc[single_df['source_'] ==0]['ID'])
        es = list(single_df.loc[single_df['source_'] ==0]['Essayset'])
        if len(test_id) != len(es):
            print('PANGA')
            break

        single_df['candi_score'] = single_df[['score_1','score_2' ,'score_3' ,'score_4' ,'score_5']].mean(axis=1)
        single_df['candi_score'] = list(map(lambda x : round(x),single_df['candi_score']))
        single_df.drop(['score_1','score_2' ,'score_3' ,'score_4' ,'score_5','ID','index'],inplace=True,axis=1)

        single_df['kitna_aacha']=single_df['candi_score']

        single_df.drop(['min_score','max_score','Essayset','candi_score'],inplace=True,axis=1)

        #all_text  = []
        
        
        
        parts_dataset = get_part_of_speech_tags(list(single_df['EssayText']))
        #ner_tags = get_ner_tags(list(single_df['EssayText']))
        #noun_p = get_noun_phrase(list(single_df['EssayText']))
        #vectors = pd.DataFrame(embed_fn(list(single_df['EssayText'])))
        punc_count = get_count_of_puncs(list(single_df['EssayText']))
        lemma_cnt =count_lemma_for_all(list(single_df['EssayText']))
        
        single_df['EssayText']=single_df['EssayText'].apply(preprocess_data)
        
        
        
        mistakes = spelling_mistakes_check(list(single_df['EssayText']))
        sen_len = get_sen_len(list(single_df['EssayText']))
        
        num_char = get_number_of_char(list(single_df['EssayText']))


        word_sent_ratio = word_to_sent_ratio(list(single_df['EssayText']))

        avg_len = get_avg_word_length_per_essay(list(single_df['EssayText']))

        
        #tfidf_vec = tfidf(list(single_df['EssayText']))
        print('vectors_fetched!!')
        #all_text = list(single_df['EssayText'])
        #vectors = get_vector(model,list(single_df['EssayText']))

        #from sklearn.feature_extraction.text import TfidfVectorizer
        #tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=20, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
        #tfidf.fit(all_text)

        #features = tfidf.transform(single_df['EssayText'])

        new_tfidf_features = pd.DataFrame()
        new_tfidf_features['clarity'] = list(single_df['clarity'])
        new_tfidf_features['sen_len'] = sen_len
        #new_tfidf_features['noun_p'] = noun_p
        new_tfidf_features['coherent'] = list(single_df['coherent'])
        new_tfidf_features['mistakes'] = mistakes
        #new_tfidf_features['lc'] = lemma_cnt
        
        new_tfidf_features['punc_count'] = punc_count
        new_tfidf_features['num_char'] = num_char
        new_tfidf_features['word_sent_r'] = word_sent_ratio
        new_tfidf_features['avg_len'] = avg_len
        
        
        
        new_tfidf_features['kitna_aacha'] = list(single_df['kitna_aacha'])
        new_tfidf_features['source_'] = list(single_df['source_'])
        
        
        new_tfidf_features = pd.concat([new_tfidf_features,parts_dataset],axis=1)
        print(new_tfidf_features.shape)
        
        new_tfidf_features.dropna(axis=0,inplace=True)
        all_preprocessed_single_q_data =  pd.get_dummies(new_tfidf_features,columns=['clarity','coherent'])
        preprocessed_main_test = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 0]
        preprocessed_main_train = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 1]

        preprocessed_main_test.drop(['source_','kitna_aacha'],axis =1,inplace=True)

        y_train = (preprocessed_main_train['kitna_aacha'])
        x_train = preprocessed_main_train.drop(['kitna_aacha','source_'],axis =1)
        
        from sklearn.model_selection import train_test_split
        from sklearn.naive_bayes import MultinomialNB,GaussianNB
        from sklearn.linear_model import LogisticRegression
        X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =0)
    
        num_cla = len(set(y_train))
        print('----')
        print(num_cla)
        
        
        
        
        params = {
'num_leaves': 60,
'max_bin': 110,
'min_data_in_leaf': 50,
'learning_rate': 0.01,
'min_sum_hessian_in_leaf': 0.000446,
'bagging_fraction': 0.60,
'bagging_freq': 15,
'max_depth': 20,
'save_binary': True,
'seed': 31452,
'feature_fraction_seed': 31415,
'feature_fraction': 0.51,
'bagging_seed': 31415,
'drop_seed': 31415,
'data_random_seed': 31415,
'objective': 'regression',
'boosting_type': 'gbdt',
'verbose': 1,
'metric': 'rmse',
'is_unbalance': False,
#     'categorical_feature': "name:branch_id"
# 'scale_pos_weight': len(train_df["target"] == 0) / sum(train_df["target"])
}

# Import the model we are using
#from sklearn.ensemble import RandomForestClassifier
# from tpot import TPOTClassifier
        import lightgbm as lgb
        print(X_train.shape)
        print(y_train_internal.shape)
        d_train = lgb.Dataset(X_train, label=y_train_internal)
# Instantiate model with 1000 decision trees
#rf = RandomForestClassifier()

# Train the model on training data
#rm = rf.fit(X_train, y_train_internal)

 
    

        #n_estimators = clf.best_iteration



        clf = lgb.train(params, d_train,30000)

        #pred_internal = rm.predict(preprocessed_main_test)
        #res =list(zip(test_id,es,pred_internal))
        #from sklearn import metrics
        if flag ==0 :
            pred_internal = clf.predict(X_test)
            print(pred_internal)
            pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],pred_internal))
            from sklearn import metrics
            pp.append(metrics.accuracy_score(y_test_internal, pred_internal))
        if flag ==1:
            pred_internal = clf.predict(preprocessed_main_test)
            #pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],pred_internal))
            res =list(zip(test_id,es,pred_internal))
            pp.append(res)
    return pp
    #res =list(zip(test_id,pred_internal))
    

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:86: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [ ]:
# from gensim.models.keyedvectors import KeyedVectors
# model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [79]:
from sklearn.model_selection import GridSearchCV
a = different_models('dcd',multi_datasets)

1.0
vectors_fetched!!
(2195, 45)


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


----
4
(1638, 49)
(1638,)
2.0
vectors_fetched!!
(1679, 43)
----
4
(1253, 47)
(1253,)
3.0
vectors_fetched!!
(2164, 45)
----
3
(1758, 49)
(1758,)
4.0
vectors_fetched!!
(1905, 44)
----
3
(1610, 48)
(1610,)
5.0
vectors_fetched!!
(2343, 43)
----
4
(1745, 47)
(1745,)
6.0
vectors_fetched!!
(2359, 45)
----
4
(1760, 49)
(1760,)
7.0
vectors_fetched!!
(2347, 43)
----
3
(1748, 47)
(1748,)
8.0
vectors_fetched!!
(2350, 44)
----
3
(1751, 48)
(1751,)
9.0
vectors_fetched!!
(2352, 44)
----
3
(1753, 48)
(1753,)
10.0
vectors_fetched!!
(2133, 44)
----
3
(1587, 48)
(1587,)


In [80]:
from itertools import chain
import itertools
l = list(itertools.chain(*a))

In [81]:
l

[(1673, 1.0, 0.887353247670009),
 (1674, 1.0, 0.6228040486059262),
 (1675, 1.0, 2.683417578841114),
 (1676, 1.0, 0.21038973747620227),
 (1677, 1.0, -0.07435227367987401),
 (1678, 1.0, 2.278907765699428),
 (1679, 1.0, 0.1896145430054261),
 (1680, 1.0, 0.8023723919200301),
 (1681, 1.0, 1.9776557823109384),
 (1682, 1.0, 2.2520263441392427),
 (1683, 1.0, 1.9655396136569998),
 (1684, 1.0, 0.9487287724274194),
 (1685, 1.0, 0.32171181017614436),
 (1686, 1.0, 1.8515066830937976),
 (1687, 1.0, 1.0141790666707335),
 (1688, 1.0, 2.2916666160796395),
 (1689, 1.0, 2.3732825643981608),
 (1690, 1.0, 0.0842344854653409),
 (1691, 1.0, 1.0590855573091265),
 (1692, 1.0, 2.377857634034379),
 (1693, 1.0, 0.5280818248985901),
 (1694, 1.0, 0.13434201548017632),
 (1695, 1.0, 0.30284318315831593),
 (1696, 1.0, 0.7657721928011895),
 (1697, 1.0, 1.0376206767043838),
 (1698, 1.0, 0.22555771011316744),
 (1699, 1.0, 2.90000650602914),
 (1700, 1.0, 2.5560939731637777),
 (1701, 1.0, 2.5165110025015207),
 (1702, 1.0, 

In [82]:
res_ = pd.DataFrame(l,columns=['id','essay_set','essay_score'])
res_n = list(map(lambda x : round(x) , list(res_['essay_score'])))
res_['essay_score'] = res_n
res_

,id,essay_set,essay_score
0,1673,1.0,1
1,1674,1.0,1
2,1675,1.0,3
3,1676,1.0,0
4,1677,1.0,0
5,1678,1.0,2
6,1679,1.0,0
7,1680,1.0,1
8,1681,1.0,2
9,1682,1.0,2


In [83]:
res_.to_csv('/home/prashant/incedo/sen_level_embed_tfidf_todu_fet_no_embed__added_regression.csv',index=False)

In [ ]:
# df_test.head()

In [ ]:
# df_test['EssayText'][22]

In [ ]:
# import operator
# l  = [[9.99690792e-01 ,1, 1.51614726e-07, 4.23026149e-13],[1,2,3,4]]
# index, value = max(enumerate(l), key=operator.itemgetter(1))

In [ ]:
# pred_internal = list(map(lambda x : max(enumerate(x), key=operator.itemgetter(1))[0],l))

In [ ]:
# doc = nlp("")
# cnt = 0
# for chunk in doc.noun_chunks:
#     cnt +=1
# print(cnt)

In [ ]:
#78.7

# 'num_leaves': 15,
# 'max_bin': 100,
# 'num_class':num_cla,
# 'min_data_in_leaf': 50,
# 'learning_rate': 0.01,
# 'min_sum_hessian_in_leaf': 0.000446,
# 'bagging_fraction': 0.55,
# 'bagging_freq': 5,
# 'max_depth': 14,
# 'save_binary': True,
# 'seed': 31452,
# 'feature_fraction_seed': 31415,
# 'feature_fraction': 0.51,
# 'bagging_seed': 31415,
# 'drop_seed': 31415,
# 'data_random_seed': 31415,
# 'objective': 'multiclass',
# 'boosting_type': 'gbdt',
# 'verbose': 1,htpo
# 'metric': 'multi_logloss',

# 'is_unbalance': False,

In [13]:
from nltk.corpus import brown
word_list = brown.words()
word_set = set(word_list)

def spelling_mistakes_check(sentences):
    for i in sentences:
        l = []
        cnt = 0
        words = i.split(' ')
        for w in words:
            if w not in word_set:
                cnt +=1
        l.append(cnt)
    return l

LookupError: 
**********************************************************************
  Resource [93mbrown[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('brown')
  [0m
  Attempted to load [93mcorpora/brown[0m

  Searched in:
    - '/home/prashant/nltk_data'
    - '/home/prashant/miniconda2/envs/p3/nltk_data'
    - '/home/prashant/miniconda2/envs/p3/share/nltk_data'
    - '/home/prashant/miniconda2/envs/p3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
/home/prashant/Downloads/essay_scoring-master/stanford-corenlp-full-2018-10-05